# Word Search Solver
I was inspired by [this Reddit post](https://www.reddit.com/r/puzzles/comments/1e5ml2e/my_wife_says_word_searches_are_too_easy_so_i_made/) to write an automated [word search](https://en.wikipedia.org/wiki/Word_search) solver. It's a great excuse to work with OCR and image processing tools.

I thought I'd need some image segmenting and clusering tools to determine the grid, but it turns out for nicely formatted grids given as a nice digital image we can actually do things much more easily with just numpy.

## Solution

### Tools

In [ ]:
from PIL import Image, ImageDraw

In [ ]:
import numpy as np

In [ ]:
import pytesseract

### Preprocessing / Transcription

In [ ]:
def axis_gridding(ax):
    streak_indices = np.argwhere(np.diff(ax)).ravel() # excluding start and end
    streak_lengths = np.diff(streak_indices)
    streak_centers = streak_indices[:-1] + streak_lengths/2
    char_centers = streak_centers[0::2]
    n = len(char_centers)
    grid_spacing, grid_offset = np.polyfit(np.arange(n), char_centers, 1)
    return n, grid_spacing, grid_offset

def transcribe_wordsearch(img, verbose=False):
    img = img.convert('L')
    arr = np.array(img)
    bin_arr = arr < 128
    
    nx, dx, ox = axis_gridding(np.any(bin_arr, axis=0))
    ny, dy, oy = axis_gridding(np.any(bin_arr, axis=1))
    if verbose:
        print(f"{nx} columns, spaced {dx}, starting at {ox}")
        print(f"{ny} rows, spaced {dy}, starting at {oy}")

    grid = []
    for i in range(ny):
        grid.append([])
        y0 = max(0, oy + i * dy - dy//2)
        y1 = min(img.size[1], oy + i * dy + dy//2)
        for j in range(nx):
            x0 = max(0, ox + j * dx - dx//2)
            x1 = min(img.size[0], ox + j * dx + dx//2)
            cimg = img.crop((x0, y0, x1, y1))
            c = pytesseract.image_to_string(cimg, config='--psm 10').strip()
            grid[-1].append(c)
            if verbose:
                print(f"Processed row {i} / {ny}, column {j} / {nx}: {c}")
    return grid, (nx, dx, ox), (ny, dy, oy)

In [ ]:
def check_grid(grid_img, nx, dx, ox, ny, dy, oy):
    check_grid_img = grid_img.copy()
    draw_check_grid_img = ImageDraw.Draw(check_grid_img)
    for i in range(nx):
        draw_check_grid_img.line((ox+dx*i,0,ox+dx*i,check_grid_img.size[1]), fill=(255,0,0), width=1)
    for i in range(nx):
        draw_check_grid_img.line((0,oy+dy*i,check_grid_img.size[0],oy+dy*i), fill=(255,0,0), width=1)
    return check_grid_img

### Search Algorithm

In [ ]:
def build_trie(wordlist):
    trie = dict()
    for word in wordlist:
        curr = trie
        for c in word.upper():
            if c not in curr:
                curr[c] = dict()
            curr = curr[c]
        if 'END' not in curr:
            curr['END'] = word
    return trie

In [ ]:
def wordsearch(grid, wordlist):
    trie = build_trie(wordlist)
    nx,ny = len(grid),len(grid[0])
    result = dict()
    for i in range(nx):
        for j in range(ny):
            for di in [-1,0,1]:
                for dj in [-1,0,1]:
                    if di == 0 and dj == 0:
                        continue
                    ip,jp = i,j
                    curr = trie
                    c = grid[i][j]
                    while c in curr:
                        curr = curr[c]
                        if 'END' in curr:
                            w = curr['END']
                            if w not in result:
                                result[w] = []
                            result[w].append(((i,j), (ip,jp)))
                        ip += di
                        jp += dj
                        if not ((0 <= ip < len(grid)) and (0 <= jp < len(grid[0]))):
                            break
                        c = grid[ip][jp]
    return result

### Display Results

In [ ]:
def draw_solution(img, solution, nx, dx, ox, ny, dy, oy, circling_size=0.8):
    solved = img.copy()
    draw_solved = ImageDraw.Draw(solved)
    red = (255,0,0)

    rx = dx / 2 * circling_size
    ry = dy / 2 * circling_size
    for locations in solution.values():
        for ((y0,x0),(y1,x1)) in locations:
            x0 = ox + dx * x0
            y0 = oy + dy * y0
            x1 = ox + dx * x1
            y1 = oy + dy * y1

            #draw_solved.line(((x0,y0),(x1,y1)), fill=(255,0,0), width=2)
            angle = np.arctan2((y1-y0), x1-x0) % (2 * np.pi)
            cdx = np.cos(angle-np.pi/2) * rx
            cdy = np.sin(angle-np.pi/2) * ry
            deg = angle * 180 / np.pi
            draw_solved.line(((x0-cdx,y0-cdy),(x1-cdx,y1-cdy)), fill=red, width=3)
            draw_solved.arc(((x1-rx,y1-ry),(x1+rx,y1+ry)), deg-90, deg+90, fill=red, width=3)
            draw_solved.line(((x0+cdx,y0+cdy),(x1+cdx,y1+cdy)), fill=red, width=3)
            draw_solved.arc(((x0-rx,y0-ry),(x0+rx,y0+ry)), deg+90, deg+90+180, fill=red, width=3)
    return solved

## Example 1: u/xuol's from Reddit
Original post [here](https://www.reddit.com/r/puzzles/comments/1e5ml2e/my_wife_says_word_searches_are_too_easy_so_i_made/).

### Open Data

In [ ]:
import requests
import io

In [ ]:
puzzle_img = Image.open(io.BytesIO(requests.get('https://i.redd.it/fg9pqcl7v3dd1.png').content))
#puzzle_img = Image.open('uxol-wordsearch.png')
puzzle_img

In [ ]:
grid_img = puzzle_img.crop((0,0,puzzle_img.size[0]-200,puzzle_img.size[1]))
grid_img.resize((grid_img.size[0]//3, grid_img.size[1]//3))

In [ ]:
wordlist_img = puzzle_img.crop((puzzle_img.size[0]-200,0,puzzle_img.size[0],puzzle_img.size[1]))
wordlist_img.resize((wordlist_img.size[0]//3, wordlist_img.size[1]//3))

In [ ]:
wordlist = pytesseract.image_to_string(wordlist_img).split()
wordlist

### Apply Solution

#### Transcribe

In [ ]:
grid, (nx, dx, ox), (ny, dy, oy) = transcribe_wordsearch(grid_img)

In [ ]:
check_grid(grid_img, nx, dx, ox, ny, dy, oy)

In [ ]:
print('\n'.join(' '.join(row) for row in grid))

#### Correct Transcription

In [ ]:
remap = {'|': 'I', 'l': 'I'}
fixedgrid = [[remap.get(c, c) for c in row] for row in grid]
print('\n'.join(' '.join(row) for row in fixedgrid))

#### Perform Search

In [ ]:
solution = wordsearch(fixedgrid, wordlist)
solution

In [ ]:
for w in wordlist:
    if w not in solution:
        print(f"{w} NOT FOUND")
    else:
        print(f"- '{w}' found {len(solution[w])} times")

### Visualize Solution

In [ ]:
draw_solution(grid_img, solution, nx, dx, ox, ny, dy, oy, circling_size=0.6)

## Example 2: u/OTDisLanguish's from Reddit
Original post [here](https://www.reddit.com/r/puzzles/comments/1e62t9s/worst_search_blame_uxuol/).

In [ ]:
puzzle_img = Image.open(io.BytesIO(requests.get('https://i.redd.it/8u47ucjmg7dd1.png').content))
puzzle_img

In [ ]:
grid_img = puzzle_img.crop((0,0,puzzle_img.size[0]-620,puzzle_img.size[1]))
grid_img.resize((grid_img.size[0]//3, grid_img.size[1]//3))

In [ ]:
wordlist_img = puzzle_img.crop((puzzle_img.size[0]-620,166,puzzle_img.size[0],puzzle_img.size[1]))
wordlist_img.resize((wordlist_img.size[0]//3, wordlist_img.size[1]//3))

In [ ]:
wordlist = pytesseract.image_to_string(wordlist_img).split()
wordlist

In [ ]:
grid, (nx, dx, ox), (ny, dy, oy) = transcribe_wordsearch(grid_img)

In [ ]:
check_grid(grid_img, nx, dx, ox, ny, dy, oy)

In [ ]:
print('\n'.join(' '.join(row) for row in grid))

In [ ]:
remap = {'Ss': 'S'}
fixedgrid = [[remap.get(c, c) for c in row] for row in grid]
print('\n'.join(' '.join(row) for row in fixedgrid))

In [ ]:
solution = wordsearch(fixedgrid, wordlist)
solution

In [ ]:
for w in wordlist:
    if w not in solution:
        print(f"- '{w}' NOT FOUND")
    else:
        print(f"- '{w}' found {len(solution[w])} times")

In [ ]:
draw_solution(grid_img, solution, nx, dx, ox, ny, dy, oy, circling_size=0.9)

## References
- https://pypi.org/project/pytesseract/
- https://en.wikipedia.org/wiki/Word_search
- https://scikit-image.org/docs/stable/user_guide/tutorial_segmentation.html
- https://scikit-image.org/docs/0.20.x/auto_examples/segmentation/plot_regionprops.html